# Get mlrpt working with the saved data

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



Need a command string in the form of `mlrpt $startT-$stopT (time HHMM in UTC) -t $durationMin`

In [3]:
latest_data <- list.files(pattern="satellite_data_*") %>% sort(decreasing = TRUE) %>% pluck(1)

In [4]:
satellite_data_df <- readRDS(latest_data)

Convert to satellite passes each day

Get the current time in UTC

In [5]:
(current_time_utc <- Sys.time() %>% .POSIXct("GMT"))

[1] "2019-09-14 19:19:58 GMT"

In [6]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

In [8]:
paste0('systemd-run --user --no-block --unit=\"mlrpt_', satellite_data_df$startDate, "_",
       satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/local/bin/mlrpt \"', satellite_data_df$startTime,"-", satellite_data_df$endTime,
       '\" -t ', satellite_data_df$duration)

[1] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-14_21:33\" --on-calendar=\"2019-09-14 21:33\" /usr/local/bin/mlrpt \"19:34-19:47\" -t 13"
 [2] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-15_07:09\" --on-calendar=\"2019-09-15 07:09\" /usr/local/bin/mlrpt \"05:10-05:24\" -t 14"
 [3] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-15_08:49\" --on-calendar=\"2019-09-15 08:49\" /usr/local/bin/mlrpt \"06:50-07:04\" -t 14"
 [4] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-15_21:12\" --on-calendar=\"2019-09-15 21:12\" /usr/local/bin/mlrpt \"19:13-19:28\" -t 15"
 [5] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-16_08:29\" --on-calendar=\"2019-09-16 08:29\" /usr/local/bin/mlrpt \"06:30-06:45\" -t 15"
 [6] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-16_20:52\" --on-calendar=\"2019-09-16 20:52\" /usr/local/bin/mlrpt \"18:53-19:09\" -t 16"
 [7] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-17_08:08\" --on-calendar=\"2019-09-17 08:08\" /usr/local/bin/mlrpt \"06:09-06:25\" -t 16"
 [8] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-17_20:32\" --on-calendar=\"2019-09-17 20:32\" /usr/local/bin/mlrpt \"18:33-18:49\" -t 16"
 [9] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-18_07:49\" --on-calendar=\"2019-09-18 07:49\" /usr/local/bin/mlrpt \"05:50-06:05\" -t 15"
[10] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-18_20:13\" --on-calendar=\"2019-09-18 20:13\" /usr/local/bin/mlrpt \"18:14-18:29\" -t 15"
[11] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_07:29\" --on-calendar=\"2019-09-19 07:29\" /usr/local/bin/mlrpt \"05:30-05:45\" -t 15"
[12] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_19:53\" --on-calendar=\"2019-09-19 19:53\" /usr/local/bin/mlrpt \"17:54-18:09\" -t 15"
[13] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-19_21:33\" --on-calendar=\"2019-09-19 21:33\" /usr/local/bin/mlrpt \"19:34-19:48\" -t 14"
[14] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_07:10\" --on-calendar=\"2019-09-20 07:10\" /usr/local/bin/mlrpt \"05:11-05:25\" -t 14"
[15] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_08:50\" --on-calendar=\"2019-09-20 08:50\" /usr/local/bin/mlrpt \"06:51-07:05\" -t 14"
[16] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_19:34\" --on-calendar=\"2019-09-20 19:34\" /usr/local/bin/mlrpt \"17:35-17:48\" -t 13"
[17] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-20_21:13\" --on-calendar=\"2019-09-20 21:13\" /usr/local/bin/mlrpt \"19:14-19:29\" -t 15"
[18] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-21_08:29\" --on-calendar=\"2019-09-21 08:29\" /usr/local/bin/mlrpt \"06:30-06:45\" -t 15"
[19] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-21_20:53\" --on-calendar=\"2019-09-21 20:53\" /usr/local/bin/mlrpt \"18:54-19:09\" -t 15"
[20] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-22_08:09\" --on-calendar=\"2019-09-22 08:09\" /usr/local/bin/mlrpt \"06:10-06:25\" -t 15"
[21] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-22_20:33\" --on-calendar=\"2019-09-22 20:33\" /usr/local/bin/mlrpt \"18:34-18:49\" -t 15"
[22] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-23_07:49\" --on-calendar=\"2019-09-23 07:49\" /usr/local/bin/mlrpt \"05:50-06:06\" -t 16"
[23] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-23_20:13\" --on-calendar=\"2019-09-23 20:13\" /usr/local/bin/mlrpt \"18:14-18:29\" -t 15"
[24] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-24_07:30\" --on-calendar=\"2019-09-24 07:30\" /usr/local/bin/mlrpt \"05:31-05:46\" -t 15"
[25] "systemd-run --user --no-block --unit=\"mlrpt_2019-09-24_19:54\" --on-calendar=\"2019-09-24 19:54\" /usr/local/bin/mlrpt \"17:55-18:09\" -t 14"

In [13]:
read_delim("timers.txt")

ERROR: Error in read_delim("timers.txt"): argument "delim" is missing, with no default
